In [1]:
import os

In [2]:
files_to_remove = os.listdir('./output/')
files_to_remove.remove('metrics.json')
for file_to_remove in files_to_remove:
    os.remove(f'./output/{file_to_remove}')

In [3]:
import matplotlib.pyplot as plt
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import torch, torchvision
import detectron2
import json
import ast 
import tensorboard,os
from detectron2.utils.logger import setup_logger

setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
# Version
data = pd.read_csv("./Data.csv").sample(frac=1)
idx = 0

In [5]:
data

,Path,XMin,YMin,XMax,YMax
4015,4437.png,0.490625,0.704971,0.528750,0.845966
2093,2336.png,0.922500,0.009700,0.999375,0.171958
3742,4153.png,0.315000,0.195000,0.334375,0.224167
5154,5620.png,0.208750,0.887041,0.280000,0.967429
4281,4718.png,0.093809,0.845000,0.182927,0.870625
...,...,...,...,...,...
4656,5109.png,0.311250,0.417195,0.381875,0.676923
562,610.png,0.257238,0.005000,0.351893,0.075000
2494,2759.png,0.405000,0.206379,0.666875,0.409006
3957,4379.png,0.620625,0.169519,0.668125,0.303993


In [6]:
# data['XMin'] = data['XMin']*255
# data['YMin'] = data['YMin']*255
# data['XMax'] = data['XMax']*255
# data['YMax'] = data['YMax']*255

In [7]:
# for i in tqdm(range(len(data))):
#     info = data.iloc[i]
# #     ymin,xmin,ymax,xmax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
#     height,width = cv2.imread(f'./Img/{info["Path"]}').shape[:2]
#     xmin,ymin,xmax,ymax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
# #     print(xmin,ymin,xmax,ymax)
#     xmin = round(xmin * width)
#     xmax = round(xmax * width)
#     ymin = round(ymin * height)
#     ymax = round(ymax * height)
#     x = xmin
#     y = ymin
#     w = xmax - xmin
#     h = ymax - ymin
# #     x,y,w,h = x*255,y*255,w*255,h*255
#     x,y,w,h = round(x),round(y),round(w),round(h)
# #     print(x,y,w,h,xmin,xmax,ymin,ymax,height,width)
#     img = cv2.imread(f'./Img/{info["Path"]}')
#     cv2.imwrite('./output.png',img)
#     roi=img[y:y+h,x:x+w]
#     cv2.imwrite(str('crop') + '.jpg', roi)
#     cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),10)
#     cv2.imwrite(str('box') + '.jpg', img)
#     plt.imshow(img)

In [8]:
info = data.iloc[50]
# xmin,ymin,xmax,ymax = info['XMin']*im[0],info['YMin']*im[1],info['XMax']*im[0],info['YMax']*im[1]
xmin,ymin,xmax,ymax = info['XMin']*255,info['YMin']*255,info['XMax']*255,info['YMax']*255
# xmin,ymin,xmax,ymax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
# x,y,w,h = x*255,y*255,w*255,h*255
# x,y,w,h = x*255,y*255,w*255,h*255
x,y,w,h = round(x),round(y),round(w),round(h)

In [9]:
# Loading Data
def load_data(data=data, test=False):
    if test is True:
        if "data.npy" in os.listdir("./"):
            data = np.load("./data.npy", allow_pickle=True)
            data = data[:325]
            print(len(data))
            return data
    if "data.npy" in os.listdir("./"):
        data = np.load("./data.npy", allow_pickle=True)
        print(len(data))
        return data
    new_data = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        height, width = cv2.imread("./Img/" + info["Path"]).shape[:2]
        xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
        xmin = round(xmin * width)
        xmax = round(xmax * width)
        ymin = round(ymin * height)
        ymax = round(ymax * height)
        record["file_name"] = "./Img/" + info["Path"]
        record["height"] = height
        record["width"] = width
        record["cateogry_id"] = 1
        objs = [
            {
                "bbox": [xmin,ymin,xmax,ymax],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": 1,
            }
        ]
        record["image_id"] = idx
        record["annotations"] = objs
        new_data.append(record)
    np.random.shuffle(new_data)
    np.save("data.npy", new_data)
    return new_data

In [10]:
# Config
labels = ["Card"]

In [11]:
# Adding the data
DatasetCatalog.register("data", lambda: load_data())
MetadataCatalog.get("data").set(thing_classes=labels)
metadata = MetadataCatalog.get("data")
DatasetCatalog.register("test", lambda: load_data(test=True))
MetadataCatalog.get("test").set(thing_classes=labels)
metadata_test = MetadataCatalog.get("test")

In [12]:
# from detectron2.utils.logger import setup_logger
# setup_logger()
# # models = [
# #     "fast_rcnn_R_50_FPN_1x.yaml",
# #     "faster_rcnn_R_50_C4_1x.yaml",
# #     "faster_rcnn_R_50_C4_3x.yaml",
# #     "faster_rcnn_R_50_DC5_1x.yaml",
# #     "faster_rcnn_R_50_DC5_3x.yaml",
# #     "retinanet_R_50_FPN_1x.py",
# #     "retinanet_R_50_FPN_1x.yaml",
# #     "retinanet_R_50_FPN_3x.yaml",
# #     "rpn_R_50_C4_1x.yaml",
# #     "rpn_R_50_FPN_1x.yaml"
# #     "faster_rcnn_R_50_FPN_1x.yaml",
# #     "faster_rcnn_R_50_FPN_3x.yaml",
# #     "faster_rcnn_R_101_DC5_3x.yaml",
# #     "faster_rcnn_R_101_FPN_3x.yaml",
# #     "faster_rcnn_X_101_32x8d_FPN_3x.yaml",
# #     "",
# # ]
# model = f"COCO-Detection/retinanet_R_101_FPN_3x.yaml"
# torch.cuda.empty_cache()
# wandb.tensorboard.patch("./output")
# wandb.init(project='Find-Card',name='baseline')
# torch.cuda.empty_cache()
# cfg = get_cfg()
# torch.cuda.empty_cache()
# cfg.merge_from_file(model_zoo.get_config_file(model))
# torch.cuda.empty_cache()
# cfg.DATASETS.TRAIN = ('data',)
# torch.cuda.empty_cache()
# cfg.DATASETS.TEST = ()
# torch.cuda.empty_cache()
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
# torch.cuda.empty_cache()
# cfg.SOLVER.MAX_ITER = 100
# torch.cuda.empty_cache()
# cfg.TEST.EVAL_PERIOD = 50
# cfg.SOLVER.BASE_LR = 0.00025
# torch.cuda.empty_cache()
# cfg.SOLVER.STEPS = []
# torch.cuda.empty_cache()
# cfg.SOLVER.IMS_PER_BATCH = 2
# torch.cuda.empty_cache()
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
# torch.cuda.empty_cache()
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
# torch.cuda.empty_cache()
# trainer = DefaultTrainer(cfg)
# torch.cuda.empty_cache()
# trainer.resume_or_load(resume=False)
# torch.cuda.empty_cache()
# trainer.train()
# torch.cuda.empty_cache()
# # cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
# torch.cuda.empty_cache()
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# torch.cuda.empty_cache()
# predictor = DefaultPredictor(cfg)
# torch.cuda.empty_cache()
# cfg.MODEL.WEIGHTS = './output/model_final.pth'
# cfg.SOLVER.SCORE_THRESH_TEST = 0.25
# predictor = DefaultPredictor(cfg)
# evaluator = COCOEvaluator('test',output_dir='./output/')
# val_loader = build_detection_test_loader(cfg,'test')
# metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
# wandb.log(metrics)
# evaluator = COCOEvaluator('data',output_dir='./output/')
# val_loader = build_detection_test_loader(cfg,'data')
# metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
# wandb.log(metrics)
# torch.cuda.empty_cache()
# logs = open('./output/metrics.json','r').read().split('\n')
# for log in tqdm(range(len(logs))):
#     res = ast.literal_eval(logs[log])
#     wandb.log(res)
# for img in os.listdir("./test_imgs/"):
#     torch.cuda.empty_cache()
#     v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
#     torch.cuda.empty_cache()
#     v = v.draw_instance_predictions(
#         predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
#     )
#     torch.cuda.empty_cache()
#     v = v.get_image()[:, :, ::-1]
#     torch.cuda.empty_cache()
#     plt.figure(figsize=(12, 6))
#     torch.cuda.empty_cache()
#     plt.imshow(v)
#     torch.cuda.empty_cache()
#     plt.savefig(f"./preds/{img}")
#     torch.cuda.empty_cache()
#     plt.close()
#     torch.cuda.empty_cache()    
# wandb.finish()

2021-10-17 14:53:09.750569: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20211017_145312-2uutsu76
wandb: Syncing run baseline


Loading config /home/indika/anaconda3/envs/myenv/lib/python3.8/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.



[10/17 14:53:26 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
  anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


[10/17 14:53:27 d2.engine.train_loop]: Starting training from iteration 0


/home/indika/anaconda3/envs/myenv/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[10/17 14:53:45 d2.utils.events]:  eta: 0:01:08  iter: 19  total_loss: 1.725  loss_cls: 1.338  loss_box_reg: 0.2968  time: 0.8910  data_time: 0.0306  lr: 4.7703e-05  max_mem: 5424M
[10/17 14:53:55 d2.utils.events]:  eta: 0:00:47  iter: 39  total_loss: 1.444  loss_cls: 1.172  loss_box_reg: 0.3934  time: 0.6875  data_time: 0.0023  lr: 9.7653e-05  max_mem: 5424M
[10/17 14:54:05 d2.utils.events]:  eta: 0:00:20  iter: 59  total_loss: 0.9723  loss_cls: 0.6755  loss_box_reg: 0.3074  time: 0.6123  data_time: 0.0118  lr: 0.0001476  max_mem: 5424M
[10/17 14:54:14 d2.utils.events]:  eta: 0:00:10  iter: 79  total_loss: 1.131  loss_cls: 0.7394  loss_box_reg: 0.404  time: 0.5737  data_time: 0.0089  lr: 0.00019755  max_mem: 5424M
[10/17 14:54:23 d2.utils.events]:  eta: 0:00:00  iter: 99  total_loss: 0.8986  loss_cls: 0.5954  loss_box_reg: 0.2946  time: 0.5463  data_time: 0.0038  lr: 0.0002475  max_mem: 5424M
[10/17 14:54:23 d2.engine.hooks]: Overall training speed: 98 iterations in 0:00:53 (0.5463 s 

RuntimeError: DataLoader worker (pid 126518) is killed by signal: Killed. 

In [ ]:
from detectron2.utils.logger import setup_logger
setup_logger()
models = [
    "fast_rcnn_R_50_FPN_1x.yaml",
    "faster_rcnn_R_50_C4_1x.yaml",
    "faster_rcnn_R_50_C4_3x.yaml",
    "faster_rcnn_R_50_DC5_1x.yaml",
    "faster_rcnn_R_50_DC5_3x.yaml",
    "retinanet_R_50_FPN_1x.py",
    "retinanet_R_50_FPN_1x.yaml",
    "retinanet_R_50_FPN_3x.yaml",
    "rpn_R_50_C4_1x.yaml",
    "rpn_R_50_FPN_1x.yaml"
    "faster_rcnn_R_50_FPN_1x.yaml",
    "faster_rcnn_R_50_FPN_3x.yaml",
    "faster_rcnn_R_101_DC5_3x.yaml",
    "faster_rcnn_R_101_FPN_3x.yaml",
    "faster_rcnn_X_101_32x8d_FPN_3x.yaml",
]
for model in models:
    model = f"COCO-Detection/retinanet_R_101_FPN_3x.yaml"
    torch.cuda.empty_cache()
    wandb.tensorboard.patch("./output")
    wandb.init(project='Find-Card',name='baseline')
    torch.cuda.empty_cache()
    cfg = get_cfg()
    torch.cuda.empty_cache()
    cfg.merge_from_file(model_zoo.get_config_file(model))
    torch.cuda.empty_cache()
    cfg.DATASETS.TRAIN = ('data',)
    torch.cuda.empty_cache()
    cfg.DATASETS.TEST = ()
    torch.cuda.empty_cache()
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    torch.cuda.empty_cache()
    cfg.SOLVER.MAX_ITER = 100
    torch.cuda.empty_cache()
    cfg.TEST.EVAL_PERIOD = 50
    cfg.SOLVER.BASE_LR = 0.00025
    torch.cuda.empty_cache()
    cfg.SOLVER.STEPS = []
    torch.cuda.empty_cache()
    cfg.SOLVER.IMS_PER_BATCH = 2
    torch.cuda.empty_cache()
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
    torch.cuda.empty_cache()
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    torch.cuda.empty_cache()
    trainer = DefaultTrainer(cfg)
    torch.cuda.empty_cache()
    trainer.resume_or_load(resume=False)
    torch.cuda.empty_cache()
    trainer.train()
    torch.cuda.empty_cache()
    # cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
    torch.cuda.empty_cache()
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    torch.cuda.empty_cache()
    predictor = DefaultPredictor(cfg)
    torch.cuda.empty_cache()
    cfg.MODEL.WEIGHTS = './output/model_final.pth'
    cfg.SOLVER.SCORE_THRESH_TEST = 0.25
    predictor = DefaultPredictor(cfg)
    evaluator = COCOEvaluator('test',output_dir='./output/')
    val_loader = build_detection_test_loader(cfg,'test')
    metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
    wandb.log(metrics)
    evaluator = COCOEvaluator('data',output_dir='./output/')
    val_loader = build_detection_test_loader(cfg,'data')
    metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
    wandb.log(metrics)
    torch.cuda.empty_cache()
    logs = open('./output/metrics.json','r').read().split('\n')
    for log in tqdm(range(len(logs))):
        res = ast.literal_eval(logs[log])
        wandb.log(res)
    for img in os.listdir("./test_imgs/"):
        torch.cuda.empty_cache()
        v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
        torch.cuda.empty_cache()
        v = v.draw_instance_predictions(
            predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
        )
        torch.cuda.empty_cache()
        v = v.get_image()[:, :, ::-1]
        torch.cuda.empty_cache()
        plt.figure(figsize=(12, 6))
        torch.cuda.empty_cache()
        plt.imshow(v)
        torch.cuda.empty_cache()
        plt.savefig(f"./preds/{img}")
        torch.cuda.empty_cache()
        plt.close()
        torch.cuda.empty_cache()    
    wandb.finish()